# BSLMM Simulation

### Bayesian method v.s. LMM

##### Bayesian method

$$
\begin{gathered}
\mathbf{y}=1_n \mu+\mathbf{X} \boldsymbol{\beta}+\boldsymbol{\varepsilon} \\
\beta_i \sim \mathrm{N}\left(0, \sigma_b^2 /(p \tau)\right)\\
\boldsymbol{\varepsilon} \sim \mathrm{MVN}_n\left(0, \mathbf{I}_n \tau^{-1}\right)\\
\end{gathered} \\
$$

##### LMM

$$
\begin{gathered}
\mathbf{y}=1_n \mu+\mathbf{X} \boldsymbol{\beta}+ \boldsymbol{u} +\boldsymbol{\varepsilon} \\
\boldsymbol{u} \sim \mathrm{MVN}\left(0, \sigma_b ^2 \tau^{-1} \mathbf{K}\right)\\
\boldsymbol{\varepsilon} \sim \mathrm{MVN}_n\left(0, \mathbf{I}_n \tau^{-1}\right)
\end{gathered} \\
$$
where $\mathbf{K} = 1/p W W ^{\top}$

The above LMM model is equivalent to the bayesian model.

$K = 1/p X X^{\top}$
$K_{tr,te} = 1/p X_{tr} X_{te} ^{\top}$

### BSLMM v.s. LMM
##### BSLMM
$$
\begin{gathered}
\mathbf{y}=1_{n} \mu+\mathbf{X} \tilde{\beta}+\mathbf{u}+\boldsymbol{\varepsilon} \\
\mathbf{u} \sim \mathbf{M V N}_{n}\left(0, \sigma_{b}^{2} \tau^{-1} \mathbf{K}\right), \\
\boldsymbol{\varepsilon} \sim \operatorname{MVN}_{n}\left(0, \tau^{-1} \mathbf{I}_{n}\right) \\
\tilde{\beta}_{i} \sim \pi \mathbf{N}\left(0, \sigma_{a}^{2} \tau^{-1}\right)+(1-\pi) \delta_{0},
\end{gathered}
$$
(hyper-)parameters, $\mu, \tau, \pi, \sigma_{a}$, and $\sigma_{b}$.

- $\mu$ and $\tau^{-1}$ control the phenotype mean and residual variance.
- $\pi$ controls the proportion of $\tilde{\boldsymbol{\beta}}$ values in (6) that are non-zero.
- $\sigma_{a}$ controls the expected magnitude of the (non-zero) $\tilde{\boldsymbol{\beta}}$.
- $\sigma_{b}$ controls the expected magnitude of the random effects $\mathbf{u}$.

##### Bayesian Linear Models

$$
\mathbf{y}=1_n \mu+ \mathbf{X} \tilde{\tilde \beta_i} +\boldsymbol{\varepsilon} \\
\tilde{\tilde \beta_i} \sim \pi \mathrm{N}\left(0,\left(\sigma_a^2+\sigma_b^2\right) /(p \tau)\right)+(1-\pi) \mathrm{N}\left(0, \sigma_b^2 /(p \tau)\right)
$$

##### LMM2
$$
\mathbf{y}=1_{n} \mu+\mathbf{X} {\beta} + \mathbf{w} +\mathbf{u}+\boldsymbol{\varepsilon} \\
\mathbf{w} \sim \mathbf{M V N}_{n}\left(0, \sigma_{b}^{2} \tau^{-1} \mathbf{W}\right), \\
\mathbf{u} \sim \mathbf{M V N}_{n}\left(0, \sigma_{b}^{2} \tau^{-1} \mathbf{K}\right), \\
\boldsymbol{\varepsilon} \sim \operatorname{MVN}_{n}\left(0, \tau^{-1} \mathbf{I}_{n}\right) \\
$$
where $\mathbf{W} = 1/p X X ^{\top}$

Then we can define $V = \sigma_{a}^{2} \tau^{-1} \mathbf{W} + \sigma_{b}^{2} \tau^{-1} \mathbf{K}$

Using weighted least square, we get $\hat \beta = (X^{\top} V^{-1}X)^{-1} X^{\top} V^ {-1} y$

Let $v = w + u$ we have $v \sim \mathbf{MVN}_{n}(0, \tau^{-1} (\sigma_a^2 W + \sigma_b^2 K) )$.

Using BLUP, we get 
$$
\begin{align}
\hat v = & E(v|y)\\
    = & \tau^{-1} (\hat \sigma_a^2 W + \hat \sigma_b^2 K) V^{-1} (y - X \hat \beta) \\
    = & \tau^{-1} (\hat \sigma_a^2 W + \hat \sigma_b^2 K) V^{-1} (\mathbf{I} - X \hat (X^{\top} V^{-1}X)^{-1} X^{\top} V^ {-1}) y
\end{align}
$$


So we can use the linear mixed model correction to correct the BSLMM
$$
H_{cv} = X (X^{\top} V^{-1}X)^{-1} X^{\top} V^ {-1} + \tau^{-1} (\hat \sigma_a^2 W + \hat \sigma_b^2 K) V^{-1} (\mathbf{I} - X \hat (X^{\top} V^{-1}X)^{-1} X^{\top} V^ {-1})
$$

$$
K = 1/p X_k X_{-k}^{\top}\\

H_{cv} = X_k (X^{\top} V^{-1}X)^{-1} X^{\top} V^ {-1} + \tau^{-1} (\hat \sigma_a^2 W + \hat \sigma_b^2 K) V^{-1} (\mathbf{I} - X \hat (X^{\top} V^{-1}X)^{-1} X^{\top} V^ {-1})
$$

And the corrected CV is
$$
\text{CV}_c = CV + H_{cv} Cov(y_{tr}, y_{te})
$$

In [18]:
from gamma_output_reader import GammaOutputReader as GOR
%load_ext autoreload
%autoreload 2

In [55]:
bv, gamma, hyper, para = GOR.read_output('./gamma_output/')

In [42]:
import pandas as pd
data = pd.read_csv('./data/10000SNPs.bimbam.csv')
X = data.iloc[:,3:].to_numpy()
p = X.shape[1]
W = 1/p * X @ X.T

In [52]:
output_path = './gamma_output/'
gor = GOR(output_path ,X, W)

In [58]:
s_a2, s_b2 = gor.get_var_component()

In [63]:
gor.get_var_component()

(0.4242869745838029, 0.8665811573602619)

In [62]:
gor.bv.var()

5331.089676332306

In [ ]:
K =  1/p * X @ X.T
